In [1]:
import sys
import os
from data import Database, ModelResultRepository, ModelResult
from benchmarks.mmul import MMULDataLoader, MMULBenchmark
from benchmarks import BenchmarkFactory, BenchmarkRunner
from models import ModelFactory, OpenAIModel, TestModel

# Settings
max_tests_per_benchmark = 10

# Inicjalizacja bazy danych
db = Database()
db.create_all_tables()
model_result_repo = ModelResultRepository(db)

# Inicjalizacja fabryk
model_factory = ModelFactory()
benchmark_factory = BenchmarkFactory()

# Rejestracja modeli
# model_factory.register_model("gpt-4o", OpenAIModel, "gpt-4o", rpm_limit=500)
# model_factory.register_model("gpt-4o-mini", OpenAIModel, "gpt-4o-mini", rpm_limit=500)
# model_factory.register_model("gpt-4-turbo", OpenAIModel, "gpt-4-turbo", rpm_limit=500)
# model_factory.register_model("gpt-4", OpenAIModel, "gpt-4", rpm_limit=500)
# model_factory.register_model("gpt-3.5-turbo", OpenAIModel, "gpt-3.5-turbo", rpm_limit=500)
model_factory.register_model("test", TestModel)

# Rejestracja benchmarków
benchmark_factory.register_benchmark("MMUL-0Shot", MMULBenchmark, model_result_repo, max_tests_per_benchmark=max_tests_per_benchmark, num_few_shot=0)
benchmark_factory.register_benchmark("MMUL-5Shot", MMULBenchmark, model_result_repo, max_tests_per_benchmark=max_tests_per_benchmark, num_few_shot=5)

# Utworzenie i uruchomienie runnera
runner = BenchmarkRunner(model_factory, benchmark_factory)
runner.run_benchmarks() 

Running benchmarks...

Running mmul-0shot benchmark for all models...
Data file already exists. Skipping download.
Data directory already exists. Skipping extraction.
Combined test data shape: (13985, 9)
Filtered test data shape: (570, 9)
Combined dev data shape: (228, 9)
Filtered dev data shape: (228, 9)

Model: test score: 0.26666666666666666

Finished mmul-0shot benchmark for all models

Running mmul-5shot benchmark for all models...
Data file already exists. Skipping download.
Data directory already exists. Skipping extraction.
Combined test data shape: (13985, 9)
Filtered test data shape: (570, 9)
Combined dev data shape: (228, 9)
Filtered dev data shape: (228, 9)

Model: test score: 0.2771929824561403

Finished mmul-5shot benchmark for all models

All benchmarks completed.
{'mmul-0shot': {'test': 0.26666666666666666}, 'mmul-5shot': {'test': 0.2771929824561403}}


In [2]:
# model = OpenAIModel("gpt-3.5-turbo")
# result = model.predict("What is the capital of France?")
# result

data_loader = MMULDataLoader()
mmlu_data = data_loader.process_data(max_tests_per_benchmark)
mmlu_data['dev']
mmlu_data['test']

Data file already exists. Skipping download.
Data directory already exists. Skipping extraction.
Combined test data shape: (13985, 9)
Filtered test data shape: (570, 9)
Combined dev data shape: (228, 9)
Filtered dev data shape: (228, 9)


,question,A,B,C,D,answer,subcategory,category,group
0,Statement 1 | If aH is an element of a factor ...,"True, True","False, False","True, False","False, True",B,abstract_algebra,math,STEM
1,Statement 1 | Every element of a group generat...,"True, True","False, False","True, False","False, True",C,abstract_algebra,math,STEM
2,Statement 1| Every function from a finite set ...,"True, True","False, False","True, False","False, True",A,abstract_algebra,math,STEM
3,Find the characteristic of the ring 2Z.,0,3,12,30,A,abstract_algebra,math,STEM
4,Which of these branches of the trigeminal nerv...,The supraorbital nerve,The infraorbital nerve,The mental nerve,None of the above,D,anatomy,health,"other (business, health, misc.)"
...,...,...,...,...,...,...,...,...,...
223,The median survival time to AIDS and death was...,Seroprevalent HIV-infected individuals,Seronegatives,Seroconverters,High-risk seronegatives,C,virology,health,"other (business, health, misc.)"
224,What is the Second Gem in Buddhism?,The Dharma,The Sangha,The Buddha,The Bodhisattva,A,world_religions,philosophy,humanities
225,"In which dynasty was the ""Mandate of Heaven"" ...",Shang,Zhou,Han,Xia,B,world_religions,philosophy,humanities
226,Which Japanese government promoted a kind of ...,Honen,Tanaka,Tokugawa,Meiji,D,world_religions,philosophy,humanities
